In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv("data/full_db_data_cleaned_utf8.csv", encoding="utf-8")
df.head()

,ID,school_link,image_url,school_name,cert,location,type,fees,location_link,phone_number,description_head,description_body,details_head,details_body,fees_list,long_location
0,0,https://egyptschools.info/school/international...,https://egyptschools.info/wp-content/uploads/2...,International Public School Rawdat El obour,IG,Al Obour,Governmental,15460,https://maps.google.com/maps?daddr=30.24019%2C...,tel:+201141583038,عن المدرسة الرسمية الدولية روضة العبور – Inter...,المدرسة الرسمية الدولية روضة العبور - Internat...,عن المدرسة الرسمية الدولية روضة العبور – Inter...,المدرسة الرسمية الدولية روضة العبور - Internat...,"[{'fees_title': 'FS 1 (رياض اطفال)', 'fees_val...",NaN
1,1,https://egyptschools.info/school/westcliff-int...,https://egyptschools.info/wp-content/uploads/2...,Westcliff International School,Thanwya,Al Maryoutia,Private,57867,https://maps.google.com/maps?daddr=29.94765%2C...,tel:01271112724,عن مدرسة ويستكليف الدولية – Westcliff Internat...,مدرسة ويستكليف الدولية - Westcliff Internation...,عن مدرسة ويستكليف الدولية – Westcliff Internat...,مدرسة ويستكليف الدولية - Westcliff Internation...,"[{'fees_title': 'KG', 'fees_value': '57,867 LE...",طريق سقارة - المريوطية - الهرم - الجيزة
2,2,https://egyptschools.info/school/kings-school-...,https://egyptschools.info/wp-content/uploads/2...,King's School The Crown,IG,October,Private,253000,https://maps.google.com/maps?daddr=29.98246%2C...,tel:01022777269,عن مدرسه كينجز سكول الدوليه – King’s School Th...,مدرسه كينجز سكول الدوليه - King's School The C...,عن مدرسه كينجز سكول الدوليه – King’s School Th...,مدرسه كينجز سكول الدوليه - King's School The C...,"[{'fees_title': 'Pre-KG', 'fees_value': '253,0...",داخل كومباوند بالم هيلز - مدينة 6 اكتوبر - الجيزة
3,3,https://egyptschools.info/school/green-hills-c...,https://egyptschools.info/wp-content/uploads/2...,Green Hills College,Thanwya,Mokattam,Private,63000,https://maps.google.com/maps?daddr=30.00422%2C...,tel:01050875380,عن مدرسة جرين هيلز كوليدج – Green Hills College,مدرسة جرين هيلز كوليدج - Green Hills College ب...,عن مدرسة جرين هيلز كوليدج – Green Hills College,مدرسة جرين هيلز كوليدج - Green Hills College ب...,"[{'fees_title': 'Pre-KG', 'fees_value': '63,00...",رقم (102) بالحي الثاني - محور ياسر رزق الرئيسي...
4,4,https://egyptschools.info/school/talae-al-amal...,https://egyptschools.info/wp-content/uploads/2...,Talae Al Amal Language School,Thanwya,New Heliopolis,Private,18000,https://maps.google.com/maps?daddr=30.12488%2C...,tel:01288877005,عن مدرسة طلائع الامل الإسلامية للغات – Talae A...,مدرسة طلائع الامل الإسلامية للغات - Talae Al A...,عن مدرسة طلائع الامل الإسلامية للغات – Talae A...,مدرسة طلائع الامل الإسلامية للغات - Talae Al A...,"[{'fees_title': 'Pre-KG', 'fees_value': '17,00...",هليوبوليس الجديدة - مدينة الشروق - القاهرة


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                593 non-null    int64 
 1   school_link       593 non-null    object
 2   image_url         593 non-null    object
 3   school_name       593 non-null    object
 4   cert              593 non-null    object
 5   location          593 non-null    object
 6   type              593 non-null    object
 7   fees              593 non-null    int64 
 8   location_link     346 non-null    object
 9   phone_number      346 non-null    object
 10  description_head  346 non-null    object
 11  description_body  346 non-null    object
 12  details_head      346 non-null    object
 13  details_body      346 non-null    object
 14  fees_list         593 non-null    object
 15  long_location     345 non-null    object
dtypes: int64(2), object(14)
memory usage: 74.3+ KB


In [4]:
columns_to_use = ["ID", "school_name", "cert", "location", "type", "fees"]
data = df[columns_to_use]
data.head()

,ID,school_name,cert,location,type,fees
0,0,International Public School Rawdat El obour,IG,Al Obour,Governmental,15460
1,1,Westcliff International School,Thanwya,Al Maryoutia,Private,57867
2,2,King's School The Crown,IG,October,Private,253000
3,3,Green Hills College,Thanwya,Mokattam,Private,63000
4,4,Talae Al Amal Language School,Thanwya,New Heliopolis,Private,18000


In [5]:
columns_to_represent = ["cert", "location", "type"]
for column in columns_to_represent:
    print(column, "unique values: ", data[column].unique())
    print("===============================================")

cert unique values:  ['IG' 'Thanwya' 'American']
location unique values:  ['Al Obour' 'Al Maryoutia' 'October' 'Mokattam' 'New Heliopolis'
 'Badr City' 'New Capital' 'Nasr City' "Hada'iq al-Zaytun"
 'Zahraa al Maadi' 'Cairo-Ismailia Desert Road' 'El Shorouk'
 'Sheikh Zayed' 'Faisal' 'Shubra Masr' 'The 1st Settlement' 'Rod El Farag'
 'Maadi' 'El Haram' "Hada'iq Al Ahram" 'The 5th Settlement' 'Heliopolis'
 'Agouza' 'Salam City' 'Bashtil' 'Imbaba' 'Dokki' 'Al Rehab' 'Katameya'
 'Helwan' 'Zamalek' 'Kerdasa' 'Madinty' 'North Giza' 'New Cairo'
 'Future City' 'Bab El Luq' 'Abbasiya' 'Mohandessin' '15-May'
 'The 3rd Settlement' 'Boulaq Abou El Ela' 'Downtown' 'Masr El Kadima'
 'Sheraton' 'El Daher' 'Ramses' 'Bab El Shaaria' 'Al Omraneya'
 "Hada'iq Al Qubbah" 'Sayyeda Zainab' 'New Helmeiya' 'Helmeiya Al Zaytun'
 'Abu Rawash' 'Shubramant' 'Gesr El Suez' 'Abu Nomros' 'Ghamrah']
type unique values:  ['Governmental' 'Private']


In [6]:
lat_lon = {
    'City': ['Al Obour', 'Al Maryoutia', 'October', 'Mokattam', 'New Heliopolis',
             'Badr City', 'New Capital', 'Nasr City', "Hada'iq al-Zaytun",
             'Zahraa al Maadi', 'Cairo-Ismailia Desert Road', 'El Shorouk',
             'Sheikh Zayed', 'Faisal', 'Shubra Masr', 'The 1st Settlement', 'Rod El Farag',
             'Maadi', 'El Haram', "Hada'iq Al Ahram", 'The 5th Settlement', 'Heliopolis',
             'Agouza', 'Salam City', 'Bashtil', 'Imbaba', 'Dokki', 'Al Rehab', 'Katameya',
             'Helwan', 'Zamalek', 'Kerdasa', 'Madinty', 'North Giza', 'New Cairo',
             'Future City', 'Bab El Luq', 'Abbasiya', 'Mohandessin', '15-May',
             'The 3rd Settlement', 'Boulaq Abou El Ela', 'Downtown', 'Masr El Kadima',
             'Sheraton', 'El Daher', 'Ramses', 'Bab El Shaaria', 'Al Omraneya',
             "Hada'iq Al Qubbah", 'Sayyeda Zainab', 'New Helmeiya', 'Helmeiya Al Zaytun',
             'Abu Rawash', 'Shubramant', 'Gesr El Suez', 'Abu Nomros', 'Ghamrah'],
    'Latitude': [30.2288224, 30.0426329, 31.08086685, 30.0163466, 31.124639, 
                 30.572633699999997, 29.95213615, 30.0521177, 30.107946432630147, 29.960511750000002, 
                 30.1490863, 30.11952805, 30.056131850653067, 30.018819286056782, 30.2005981, 30.0645714,
                 30.0732299, 29.95891452548896, 29.850963999999998, 29.96909, 30.003804549999998,
                 30.090123900000002, 30.0549435, 30.17479935, 30.09156495, 30.088512, 
                 30.038186465234222 , 27.2599794, 29.9874449, 29.8500001, 29.9289553, 30.031422550000002,
                 30.091736849999997, 29.9401222, 30.0277688, 30.04076775, 30.0458315, 30.066667,
                 30.05839515, 29.8541469, 29.980896700000002, 30.05544175, 30.0445133, 30.0072008,
                 30.10062255, 30.06207115, 30.0629785, 30.0551319, 30.007087, 30.0821548, 30.0273241,
                 30.1081344, 29.1797446, 30.0323187, 29.9407683, 30.0883271, 29.94945675, 30.06207115],
    'Longitude': [31.465685617766244, 31.124639, 29.724904831435097, 31.2804433, 31.6925603,
                  30.703264117160842, 31.712935437872876, 31.3422045, 31.311177992306487, 31.32331893333732,
                  31.4249067, 31.60687795516707, 30.972691160621633, 31.32421818804972, 31.3290642,
                  31.447989244439903, 31.2363328, 31.2605343965376, 31.22094695, 31.0973093,
                  31.424935726802204, 31.326542634536953, 31.2127281, 31.4091625423734, 
                  31.182251953163934, 31.215927, 31.21081926315397, 31.346871748275863, 31.5102943234146,
                  31.333333, 30.934665072489047, 31.115599860932907, 31.638060827436064, 
                  31.151556, 31.4756825, 30.982470847844823, 31.239586, 31.283333, 31.202023080983963,
                  31.385699787290086, 31.4347388067753, 31.230943570544436, 31.235728, 31.229366,
                  31.377136500620935, 31.2548915439843, 31.2460533, 31.2576045, 31.2060355,
                  31.2825119, 31.2404117, 31.3357763, 31.1542228, 31.0757089, 31.1901228, 31.3019009,
                  31.216428103427866, 31.2548915439843]
}

city_df = pd.DataFrame(lat_lon)

In [7]:
data = data.merge(city_df, how='left', left_on='location', right_on='City')

In [8]:
data.drop(columns=["City"], inplace=True)

In [9]:
data

,ID,school_name,cert,location,type,fees,Latitude,Longitude
0,0,International Public School Rawdat El obour,IG,Al Obour,Governmental,15460,30.228822,31.465686
1,1,Westcliff International School,Thanwya,Al Maryoutia,Private,57867,30.042633,31.124639
2,2,King's School The Crown,IG,October,Private,253000,31.080867,29.724905
3,3,Green Hills College,Thanwya,Mokattam,Private,63000,30.016347,31.280443
4,4,Talae Al Amal Language School,Thanwya,New Heliopolis,Private,18000,31.124639,31.692560
...,...,...,...,...,...,...,...,...
588,588,Own Heliopolis Language School,Thanwya,Sheraton,Private,14507,30.100623,31.377137
589,589,Pioneers Language School,Thanwya,October,Governmental,20000,31.080867,29.724905
590,590,Masters Language School,Thanwya,October,Private,38500,31.080867,29.724905
591,591,Solaimaneyah International School,American,October,Private,63800,31.080867,29.724905


In [10]:
cert_mapping = {'IG': 2, 'Thanwya': 0, 'American': 1}
type_mapping = {'Governmental': 0, 'Private': 1}
data['cert'] = data['cert'].map(cert_mapping)
data['type'] = data['type'].map(type_mapping)
data.head()

,ID,school_name,cert,location,type,fees,Latitude,Longitude
0,0,International Public School Rawdat El obour,2,Al Obour,0,15460,30.228822,31.465686
1,1,Westcliff International School,0,Al Maryoutia,1,57867,30.042633,31.124639
2,2,King's School The Crown,2,October,1,253000,31.080867,29.724905
3,3,Green Hills College,0,Mokattam,1,63000,30.016347,31.280443
4,4,Talae Al Amal Language School,0,New Heliopolis,1,18000,31.124639,31.692560


In [11]:
scaler = StandardScaler()
data['fees_normalized'] = scaler.fit_transform(data[['fees']])
data["Latitude_normalized"] = scaler.fit_transform(data[['Latitude']])
data["Longitude_normalized"] = scaler.fit_transform(data[['Longitude']])

In [14]:
data.to_csv("model_data.csv", index=False)

In [ ]:
# def recommend_schools(school_ids, num_recommendations=10):
#     queried_schools = data[data['ID'].isin(school_ids)]
    
#     queried_features = queried_schools[['cert', 'type', 'fees_normalized', 'Latitude_normalized', 'Longitude_normalized']].mean(axis=0).values
    
#     features = data[['cert', 'type', 'fees_normalized', 'Latitude_normalized', 'Longitude_normalized']].values
    
#     similarity = cosine_similarity(features, queried_features.reshape(1, -1))
    
#     data['similarity'] = similarity.flatten()
#     recommended_schools = data.sort_values(by='similarity', ascending=False).head(num_recommendations)
    
#     return recommended_schools[['ID', 'school_name', 'cert', "location", 'type', 'fees', 'Latitude', 'Longitude']]

# recommendations = recommend_schools([15, 37, 82])
# recommendations

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Load English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Assuming 'data' is your DataFrame containing school information

def recommend_schools(school_ids, num_recommendations=10):
    queried_schools = data[data['ID'].isin(school_ids)]
    
    # Tokenize and vectorize queried school names
    queried_school_names = queried_schools['school_name'].tolist()
    queried_school_tokens = [nlp(name) for name in queried_school_names]
    queried_school_vectors = [token.vector for token in queried_school_tokens]
    queried_school_vectors_mean = pd.DataFrame(queried_school_vectors).mean(axis=0).values
    
    queried_features = queried_schools[['cert', 'type', 'fees_normalized', 'Latitude_normalized', 'Longitude_normalized']].mean(axis=0).values
    
    features = data[['cert', 'type', 'fees_normalized', 'Latitude_normalized', 'Longitude_normalized']].values
    
    similarity = cosine_similarity(features, queried_features.reshape(1, -1))

    name_similarity = cosine_similarity([queried_school_vectors_mean], [token.vector for token in nlp.pipe(data['school_name'])])
    
    data['similarity'] = similarity.flatten() + name_similarity.flatten()
    recommended_schools = data.sort_values(by='similarity', ascending=False).head(num_recommendations)
    
    return recommended_schools[['ID', 'school_name', 'cert', "location", 'type', 'fees', 'Latitude', 'Longitude']]

recommendations = recommend_schools([500])
recommendations